In [2]:
import pandas as pd
from numpy import nan
import pandas_gbq
import altair as alt
import matplotlib as mpl
import pydeck as pdk
import json

/home/pedro/Modelos/Faculdade/hackaton-fgv/venv/lib/python3.11/site-packages/geopandas/_compat.py:153: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  set_use_pygeos()


In [3]:
query = """SELECT *
    FROM rj-rioaguas.saneamento_drenagem.ponto_supervisionado_alagamento""" # Coloquei um limite pra não gastar 3GB do google quando for puxar
drenagem = pandas_gbq.read_gbq(query, project_id = "hackaton-fgv")
drenagem

/home/pedro/Modelos/Faculdade/hackaton-fgv/venv/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/pedro/Modelos/Faculdade/hackaton-fgv/venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2257: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,id_ponto_supervisionado,classe,top_50,causa_alagamento,medida_cor,eliminado,endereco_ponto_supervisionado,bairro,area_planejamento,regiao_administrativa,bacia_hidrografica,sub_bacia_hidrografica,latitude,longitude,utm_x,utm_y,geometry,projeto,data_atualizacao,ano_vigencia
0,71,Ponto critico,<NA>,Pdmap (37d010) indica adequacao da calha e de...,Obra de macrodrenagem,None,Rua uranos,Bonsucesso,3.1,X - Ramos,G,Canal do cunha,-22.873488,-43.252871,679234.426845,7.469423e+06,POINT(-43.25287067 -22.87348768),(PDMAP 37D010) (Relatorio 53D239),2022-01-01,2022
1,16,Ponto critico,<NA>,"Alagamentos nas pistas lateral e central, sent...",Obra de microdrenagem,None,Avenida brasil (bonsucesso),Manguinhos,3.1,X - Ramos,G,Canal do cunha,-22.870685,-43.245211,680024.033531,7.469724e+06,POINT(-43.24521149 -22.87068547),_,2022-01-01,2022
2,87,Ponto critico,34,"Ponto baixo, esta recebendo obras de microdren...",Obra de microdrenagem,None,Av borges de medeiros (entre ruas gal garzon e...,Lagoa,2.1,VI - Lagoa,Zs,Lagoa rodrigo de freitas,-22.972863,-43.217383,682742.427286,7.458375e+06,POINT(-43.21738326 -22.9728625),_,2022-01-01,2022
3,88,Ponto critico,35,Insuficiencia das redes de micro e macrodrenag...,Obra de macrodrenagem,None,Rua jardim botanico / rua gal garzon,Jardim botanico,2.1,VI - Lagoa,Zs,Lagoa rodrigo de freitas,-22.967434,-43.220264,682454.295987,7.458979e+06,POINT(-43.22026438 -22.96743433),(Projeto 33D2797),2022-01-01,2022
4,90,Ponto critico,<NA>,"Prever limpeza e desassoreamento de ralos, ram...",Limpeza e obra de microdrenagem,None,Praca santos dumont,Gavea,2.1,VI - Lagoa,Zs,Lagoa rodrigo de freitas,-22.973479,-43.225926,681865.703191,7.458317e+06,POINT(-43.22592556 -22.97347927),(Cadastro 2-3-D-1099),2022-01-01,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,206,Ponto monitorado,<NA>,,_,2010,Canal do portelo (comunidade amigos do fontela),Vargem pequena,4,XXIV - Barra da Tijuca,J,Lagoa de jacarepagua,-22.993565,-43.445398,659339.200092,7.456348e+06,POINT(-43.44539816 -22.99356461),_,2022-01-01,2022
343,107,Ponto monitorado,<NA>,,_,2011,Rio muzema (estrada de jacarepagua / av eng so...,Itanhanga_,4,XXIV - Barra da Tijuca,J,Lagoa da tijuca,-22.990097,-43.320518,672145.715403,7.456591e+06,POINT(-43.32051768 -22.99009719),_,2022-01-01,2022
344,224,Ponto monitorado,<NA>,,_,2012,Rio itanhanga,Itanhanga,4,XXIV - Barra da Tijuca,J,Rio da cachoeira,-22.992669,-43.309602,673261.537876,7.456293e+06,POINT(-43.30960171 -22.99266877),_,2022-01-01,2022
345,237,Ponto monitorado,<NA>,,_,<2010,Rio grande (trecho cidade de deus),Cidade de deus,4,XXXIV - Cidade de Deus,J,Rio grande,-22.949301,-43.365383,667596.495466,7.461160e+06,POINT(-43.36538281 -22.94930057),_,2022-01-01,2022


In [4]:
drenagem["classe"].value_counts()

classe
Ponto critico       164
Ponto observado      93
Ponto monitorado     90
Name: count, dtype: int64

In [13]:
drenagem["causa_alagamento"].value_counts()

causa_alagamento
                                                                                                                                                                                                                                                           98
O rio das pedras e insuficiente no trecho.  o pdmap preve a implantacao de reservatorios de amortecimento para adequacao das vazoes a sua calha.  pdmap 37d009.                                                                                             2
Ponto solucionado com a realizacao das obras do rio pechincha                                                                                                                                                                                               2
Registro de alagamentos no sistema comando, do cor. necessaria uma investigacao do problema                                                                                                                                  

In [12]:
drenagem[drenagem["classe"] == "Ponto observado"]["eliminado"].value_counts()

eliminado
2012    1
2011    1
Name: count, dtype: int64

In [7]:
classes = {
    "Ponto critico": [0, 0, 190],
    "Ponto monitorado": [0, 255, 0],
    "Ponto observado": [100, 100, 255]
}
drenagem["coloracao"] = drenagem.apply(lambda linha: classes[linha["classe"]], axis=1)
drenagem

,id_ponto_supervisionado,classe,top_50,causa_alagamento,medida_cor,eliminado,endereco_ponto_supervisionado,bairro,area_planejamento,regiao_administrativa,...,sub_bacia_hidrografica,latitude,longitude,utm_x,utm_y,geometry,projeto,data_atualizacao,ano_vigencia,coloracao
0,71,Ponto critico,<NA>,Pdmap (37d010) indica adequacao da calha e de...,Obra de macrodrenagem,None,Rua uranos,Bonsucesso,3.1,X - Ramos,...,Canal do cunha,-22.873488,-43.252871,679234.426845,7.469423e+06,POINT(-43.25287067 -22.87348768),(PDMAP 37D010) (Relatorio 53D239),2022-01-01,2022,"[0, 0, 190]"
1,16,Ponto critico,<NA>,"Alagamentos nas pistas lateral e central, sent...",Obra de microdrenagem,None,Avenida brasil (bonsucesso),Manguinhos,3.1,X - Ramos,...,Canal do cunha,-22.870685,-43.245211,680024.033531,7.469724e+06,POINT(-43.24521149 -22.87068547),_,2022-01-01,2022,"[0, 0, 190]"
2,87,Ponto critico,34,"Ponto baixo, esta recebendo obras de microdren...",Obra de microdrenagem,None,Av borges de medeiros (entre ruas gal garzon e...,Lagoa,2.1,VI - Lagoa,...,Lagoa rodrigo de freitas,-22.972863,-43.217383,682742.427286,7.458375e+06,POINT(-43.21738326 -22.9728625),_,2022-01-01,2022,"[0, 0, 190]"
3,88,Ponto critico,35,Insuficiencia das redes de micro e macrodrenag...,Obra de macrodrenagem,None,Rua jardim botanico / rua gal garzon,Jardim botanico,2.1,VI - Lagoa,...,Lagoa rodrigo de freitas,-22.967434,-43.220264,682454.295987,7.458979e+06,POINT(-43.22026438 -22.96743433),(Projeto 33D2797),2022-01-01,2022,"[0, 0, 190]"
4,90,Ponto critico,<NA>,"Prever limpeza e desassoreamento de ralos, ram...",Limpeza e obra de microdrenagem,None,Praca santos dumont,Gavea,2.1,VI - Lagoa,...,Lagoa rodrigo de freitas,-22.973479,-43.225926,681865.703191,7.458317e+06,POINT(-43.22592556 -22.97347927),(Cadastro 2-3-D-1099),2022-01-01,2022,"[0, 0, 190]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,206,Ponto monitorado,<NA>,,_,2010,Canal do portelo (comunidade amigos do fontela),Vargem pequena,4,XXIV - Barra da Tijuca,...,Lagoa de jacarepagua,-22.993565,-43.445398,659339.200092,7.456348e+06,POINT(-43.44539816 -22.99356461),_,2022-01-01,2022,"[0, 255, 0]"
343,107,Ponto monitorado,<NA>,,_,2011,Rio muzema (estrada de jacarepagua / av eng so...,Itanhanga_,4,XXIV - Barra da Tijuca,...,Lagoa da tijuca,-22.990097,-43.320518,672145.715403,7.456591e+06,POINT(-43.32051768 -22.99009719),_,2022-01-01,2022,"[0, 255, 0]"
344,224,Ponto monitorado,<NA>,,_,2012,Rio itanhanga,Itanhanga,4,XXIV - Barra da Tijuca,...,Rio da cachoeira,-22.992669,-43.309602,673261.537876,7.456293e+06,POINT(-43.30960171 -22.99266877),_,2022-01-01,2022,"[0, 255, 0]"
345,237,Ponto monitorado,<NA>,,_,<2010,Rio grande (trecho cidade de deus),Cidade de deus,4,XXXIV - Cidade de Deus,...,Rio grande,-22.949301,-43.365383,667596.495466,7.461160e+06,POINT(-43.36538281 -22.94930057),_,2022-01-01,2022,"[0, 255, 0]"


In [8]:
def arruma_descricao(linha):
    if linha["causa_alagamento"] == " " or linha["causa_alagamento"] is None:
        return "Não foi fornecida descrição para esse ponto"
    else:
        return linha["causa_alagamento"]
drenagem["descricao"] = drenagem.apply(arruma_descricao, axis=1)
drenagem

,id_ponto_supervisionado,classe,top_50,causa_alagamento,medida_cor,eliminado,endereco_ponto_supervisionado,bairro,area_planejamento,regiao_administrativa,...,latitude,longitude,utm_x,utm_y,geometry,projeto,data_atualizacao,ano_vigencia,coloracao,descricao
0,71,Ponto critico,<NA>,Pdmap (37d010) indica adequacao da calha e de...,Obra de macrodrenagem,None,Rua uranos,Bonsucesso,3.1,X - Ramos,...,-22.873488,-43.252871,679234.426845,7.469423e+06,POINT(-43.25287067 -22.87348768),(PDMAP 37D010) (Relatorio 53D239),2022-01-01,2022,"[0, 0, 190]",Pdmap (37d010) indica adequacao da calha e de...
1,16,Ponto critico,<NA>,"Alagamentos nas pistas lateral e central, sent...",Obra de microdrenagem,None,Avenida brasil (bonsucesso),Manguinhos,3.1,X - Ramos,...,-22.870685,-43.245211,680024.033531,7.469724e+06,POINT(-43.24521149 -22.87068547),_,2022-01-01,2022,"[0, 0, 190]","Alagamentos nas pistas lateral e central, sent..."
2,87,Ponto critico,34,"Ponto baixo, esta recebendo obras de microdren...",Obra de microdrenagem,None,Av borges de medeiros (entre ruas gal garzon e...,Lagoa,2.1,VI - Lagoa,...,-22.972863,-43.217383,682742.427286,7.458375e+06,POINT(-43.21738326 -22.9728625),_,2022-01-01,2022,"[0, 0, 190]","Ponto baixo, esta recebendo obras de microdren..."
3,88,Ponto critico,35,Insuficiencia das redes de micro e macrodrenag...,Obra de macrodrenagem,None,Rua jardim botanico / rua gal garzon,Jardim botanico,2.1,VI - Lagoa,...,-22.967434,-43.220264,682454.295987,7.458979e+06,POINT(-43.22026438 -22.96743433),(Projeto 33D2797),2022-01-01,2022,"[0, 0, 190]",Insuficiencia das redes de micro e macrodrenag...
4,90,Ponto critico,<NA>,"Prever limpeza e desassoreamento de ralos, ram...",Limpeza e obra de microdrenagem,None,Praca santos dumont,Gavea,2.1,VI - Lagoa,...,-22.973479,-43.225926,681865.703191,7.458317e+06,POINT(-43.22592556 -22.97347927),(Cadastro 2-3-D-1099),2022-01-01,2022,"[0, 0, 190]","Prever limpeza e desassoreamento de ralos, ram..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,206,Ponto monitorado,<NA>,,_,2010,Canal do portelo (comunidade amigos do fontela),Vargem pequena,4,XXIV - Barra da Tijuca,...,-22.993565,-43.445398,659339.200092,7.456348e+06,POINT(-43.44539816 -22.99356461),_,2022-01-01,2022,"[0, 255, 0]",Não foi fornecida descrição para esse ponto
343,107,Ponto monitorado,<NA>,,_,2011,Rio muzema (estrada de jacarepagua / av eng so...,Itanhanga_,4,XXIV - Barra da Tijuca,...,-22.990097,-43.320518,672145.715403,7.456591e+06,POINT(-43.32051768 -22.99009719),_,2022-01-01,2022,"[0, 255, 0]",Não foi fornecida descrição para esse ponto
344,224,Ponto monitorado,<NA>,,_,2012,Rio itanhanga,Itanhanga,4,XXIV - Barra da Tijuca,...,-22.992669,-43.309602,673261.537876,7.456293e+06,POINT(-43.30960171 -22.99266877),_,2022-01-01,2022,"[0, 255, 0]",Não foi fornecida descrição para esse ponto
345,237,Ponto monitorado,<NA>,,_,<2010,Rio grande (trecho cidade de deus),Cidade de deus,4,XXXIV - Cidade de Deus,...,-22.949301,-43.365383,667596.495466,7.461160e+06,POINT(-43.36538281 -22.94930057),_,2022-01-01,2022,"[0, 255, 0]",Não foi fornecida descrição para esse ponto


In [9]:
def arruma_medida(linha):
    if linha["medida_cor"] == "_" or linha["medida_cor"] is None:
        return "Não foi fornecida medida planejada ou tomada para esse ponto"
    else:
        return linha["medida_cor"]
drenagem["medida"] = drenagem.apply(arruma_medida, axis=1)
drenagem

,id_ponto_supervisionado,classe,top_50,causa_alagamento,medida_cor,eliminado,endereco_ponto_supervisionado,bairro,area_planejamento,regiao_administrativa,...,longitude,utm_x,utm_y,geometry,projeto,data_atualizacao,ano_vigencia,coloracao,descricao,medida
0,71,Ponto critico,<NA>,Pdmap (37d010) indica adequacao da calha e de...,Obra de macrodrenagem,None,Rua uranos,Bonsucesso,3.1,X - Ramos,...,-43.252871,679234.426845,7.469423e+06,POINT(-43.25287067 -22.87348768),(PDMAP 37D010) (Relatorio 53D239),2022-01-01,2022,"[0, 0, 190]",Pdmap (37d010) indica adequacao da calha e de...,Obra de macrodrenagem
1,16,Ponto critico,<NA>,"Alagamentos nas pistas lateral e central, sent...",Obra de microdrenagem,None,Avenida brasil (bonsucesso),Manguinhos,3.1,X - Ramos,...,-43.245211,680024.033531,7.469724e+06,POINT(-43.24521149 -22.87068547),_,2022-01-01,2022,"[0, 0, 190]","Alagamentos nas pistas lateral e central, sent...",Obra de microdrenagem
2,87,Ponto critico,34,"Ponto baixo, esta recebendo obras de microdren...",Obra de microdrenagem,None,Av borges de medeiros (entre ruas gal garzon e...,Lagoa,2.1,VI - Lagoa,...,-43.217383,682742.427286,7.458375e+06,POINT(-43.21738326 -22.9728625),_,2022-01-01,2022,"[0, 0, 190]","Ponto baixo, esta recebendo obras de microdren...",Obra de microdrenagem
3,88,Ponto critico,35,Insuficiencia das redes de micro e macrodrenag...,Obra de macrodrenagem,None,Rua jardim botanico / rua gal garzon,Jardim botanico,2.1,VI - Lagoa,...,-43.220264,682454.295987,7.458979e+06,POINT(-43.22026438 -22.96743433),(Projeto 33D2797),2022-01-01,2022,"[0, 0, 190]",Insuficiencia das redes de micro e macrodrenag...,Obra de macrodrenagem
4,90,Ponto critico,<NA>,"Prever limpeza e desassoreamento de ralos, ram...",Limpeza e obra de microdrenagem,None,Praca santos dumont,Gavea,2.1,VI - Lagoa,...,-43.225926,681865.703191,7.458317e+06,POINT(-43.22592556 -22.97347927),(Cadastro 2-3-D-1099),2022-01-01,2022,"[0, 0, 190]","Prever limpeza e desassoreamento de ralos, ram...",Limpeza e obra de microdrenagem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,206,Ponto monitorado,<NA>,,_,2010,Canal do portelo (comunidade amigos do fontela),Vargem pequena,4,XXIV - Barra da Tijuca,...,-43.445398,659339.200092,7.456348e+06,POINT(-43.44539816 -22.99356461),_,2022-01-01,2022,"[0, 255, 0]",Não foi fornecida descrição para esse ponto,Não foi fornecida medida planejada ou tomada p...
343,107,Ponto monitorado,<NA>,,_,2011,Rio muzema (estrada de jacarepagua / av eng so...,Itanhanga_,4,XXIV - Barra da Tijuca,...,-43.320518,672145.715403,7.456591e+06,POINT(-43.32051768 -22.99009719),_,2022-01-01,2022,"[0, 255, 0]",Não foi fornecida descrição para esse ponto,Não foi fornecida medida planejada ou tomada p...
344,224,Ponto monitorado,<NA>,,_,2012,Rio itanhanga,Itanhanga,4,XXIV - Barra da Tijuca,...,-43.309602,673261.537876,7.456293e+06,POINT(-43.30960171 -22.99266877),_,2022-01-01,2022,"[0, 255, 0]",Não foi fornecida descrição para esse ponto,Não foi fornecida medida planejada ou tomada p...
345,237,Ponto monitorado,<NA>,,_,<2010,Rio grande (trecho cidade de deus),Cidade de deus,4,XXXIV - Cidade de Deus,...,-43.365383,667596.495466,7.461160e+06,POINT(-43.36538281 -22.94930057),_,2022-01-01,2022,"[0, 255, 0]",Não foi fornecida descrição para esse ponto,Não foi fornecida medida planejada ou tomada p...


In [16]:
camera_inicial = pdk.ViewState(
    latitude = -22.9035, 
    longitude = -43.4096, 
    zoom = 9.3, 
    max_zoom = 16, 
    pitch = 0, 
    bearing = 0
)

base = pdk.Layer(
    "PolygonLayer",
    stroked = False,
    get_polygon = "-",
    get_fill_color = [0, 0, 0, 20],
)

drenagens_nao_resolvidas = pdk.Layer(
    "ScatterplotLayer",
    drenagem[["latitude", "longitude", "coloracao", "endereco_ponto_supervisionado", "bairro", 
        "sub_bacia_hidrografica", "medida", "descricao", "eliminado"]][drenagem["classe"] == "Ponto critico" | drenagem["classe"] == "Ponto observado"],
    get_position = ["longitude", "latitude"],
    auto_highlight = True,
    get_radius = 200,
    get_fill_color = "coloracao",
    pickable = True
)

drenagens_resolvidas = pdk.Layer(
    "ScatterplotLayer",
    drenagem[["latitude", "longitude", "coloracao", "endereco_ponto_supervisionado", "bairro", 
        "sub_bacia_hidrografica", "medida", "descricao", "eliminado"]][drenagem["classe"] == "Ponto monitorado"],
    get_position = ["longitude", "latitude"],
    auto_highlight = True,
    get_radius = 200,
    get_fill_color = "coloracao",
    pickable = True
)


mapa = pdk.Deck(
    layers = [base, drenagens], 
    initial_view_state = camera_inicial,
    tooltip = {"html": "<b>Endereço:</b> {endereco_ponto_supervisionado} - {bairro}"\
               "<br><b>Sub bacia hidrografica:</b> {sub_bacia_hidrografica}<br>"\
               "<b>Descrição do ponto:</b> {descricao}<br><b>Medida planejada:</b> {medida}"\
               "<br><b>Eliminado em:</b> {eliminado}",
               "style": {
                  "backgroundColor": "steelblue",
                  "color": "white"
                        }
              },
    map_style = "dark_no_labels"
)

mapa.to_html("geojson_layer.html")

TypeError: Cannot perform 'ror_' with a dtyped [object] array and scalar of type [bool]

In [4]:
with open("Sub_Bacias_Hidrograficas.geojson","r") as f:
    bacias_rj = json.load(f)
bacias_rj

{'type': 'FeatureCollection',
 'name': 'Sub_Bacias_Hidrograficas',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'objectid': 1,
    'shi_cd': 1,
    'shi_nm': ' ',
    'shi_nm_macro': 'Água',
    'shi_version': 1,
    'shi_ar': 861.5442,
    'st_areashape': 8615465.347109426,
    'st_perimetershape': 49512.920090756656},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-43.36604265017636, -22.975905191319548],
      [-43.36606864701467, -22.975908981399538],
      [-43.36609216290723, -22.975900012323372],
      [-43.36611582823443, -22.975894837763693],
      [-43.36613898749413, -22.975891193236727],
      [-43.36615829367301, -22.975883535385933],
      [-43.36617664092302, -22.97587451301128],
      [-43.36619469234923, -22.97586575851181],
      [-43.366213444870034, -22.975855476887173],
      [-43.366231909188784, -22.97584482924678],
      [-43.36625998270187, -22.975829946731736]